# **Enable GPU**

To enable the **GPU** on the notebook, select the following menu options.

`Runtime / Change runtime type`

Select **GPU** under **Hardware Accelerator** and the notebook will use the free **GPU** provided in the cloud during processing.


In [ ]:
!nvidia-smi

# **Connect Google Colab with Google Drive**

Mount on Google Drive using the following code below. Account authorization must be confirmed.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Change the directory to the root folder of the custom dataset.

In [ ]:
%cd /content/drive/MyDrive/YOLOv8_Custom_Dataset

View a list of the contents of the directory.

**Note:** If the dataset is already extracted, it is not necessary to use the command below to re-extract the dataset.

In [ ]:
!ls

Extract the dataset file to the root directory of Google Drive.

**Note:** This process probably takes a few minutes. The time depends on the size of the dataset.

In [ ]:
!unzip "Data.zip" -d "Data"

# **Setup Ultralytics**

Install the **ultralytics** library with dependencies and check that everything is working properly.

**Note:** 
If the lib. version is not specified, the latest version is installed.

More information about the **ultralytics** library can be found at:
https://github.com/ultralytics/ultralytics

In [ ]:
!pip install ultralytics

# Ultralytics (Real-time object detection and image segmentation 
# model) [pip install ultralytics]
import ultralytics

# After a successful installation, we can use the check function to 
# see if everything is installed correctly or not.
ultralytics.checks()

# Initialization of Constants

Initialization of global constants to declare a specific dataset for training, validation and testing.

In [ ]:
"""
Description:
    Initialization of constants.
"""
# The ID of the object to be scanned.
#   ID{0} = 'T_Joint'
#   ID{1} = 'Metal_Blank'
CONST_OBJECT_ID = 0
# The identification number of the dataset type.
CONST_DATASET_TYPE = 0
# Name of the dataset.
CONST_DATASET_NAME = F'Dataset_Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}'
# Select the desired size of YOLO* to build the model (note: detection model).
#   Nano: 'yolov8n', Small: 'yolov8s', Medium: 'yolov8m', Large: 'yolov8l', XLarge: 'yolov8x'}
CONST_YOLO_SIZE = 'yolov8s'
# The root folder of the custom dataset (on Google Drive).
CONST_GOOGLE_DRIVE_ROOT = '/content/drive/MyDrive/YOLOv8_Custom_Dataset'

# Training

Training the YOLOv8 model on a custom dataset. In this case, the model is trained using the specified dataset and hyperparameters. 

For more information see: https://docs.ultralytics.com/modes/train/#arguments

In [ ]:
# Ultralytics (Real-time object detection and image segmentation 
# model) [pip install ultralytics]
from ultralytics import YOLO

def main():
  # Load a pre-trained YOLO model.
  model = YOLO(f'{CONST_YOLO_SIZE}.pt')

  # Training the model on a custom dataset with additional dependencies (number of epochs, image size, etc.)
  model.train(data=f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Configuration/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/config.yaml', batch=32, imgsz=640, epochs=100, 
              name=f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Result/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/train')

if __name__ == '__main__':
    main()

# Validation

Validation of the YOLOv8 model after training. In this case, the model is evaluated on a validation dataset to measure its accuracy and generalization performance.

For more information see: https://docs.ultralytics.com/modes/val/

In [ ]:
# Ultralytics (Real-time object detection and image segmentation 
# model) [pip install ultralytics]
from ultralytics import YOLO

def main():
  # Load a pre-trained custom YOLO model.
  model = YOLO(f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Result/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/train/weights/best.pt')

  # Evaluate the performance of the model on the validation dataset.
  model.val(data=f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Configuration/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/config.yaml', batch=32, imgsz=640,
            name=f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Result/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/valid')

if __name__ == '__main__':
    main()

# Test

Prediction (testing) using the trained YOLOv8 model on new images. In this case, the model is loaded from a checkpoint file and the user can provide images to perform inference. The model predicts the classes and locations of objects in the input images.

For more information see: https://docs.ultralytics.com/modes/predict/

In [ ]:
# Ultralytics (Real-time object detection and image segmentation 
# model) [pip install ultralytics]
from ultralytics import YOLO

def main():
  # Load a pre-trained custom YOLO model.
  model = YOLO(f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Result/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/train/weights/best.pt')

  # Predict (test) the model on a test dataset.
  model.predict(source=f'{CONST_GOOGLE_DRIVE_ROOT}/Data/Dataset_Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/images/test', save=True, imgsz=640, conf=0.5,
                name=f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Result/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/predict')

if __name__ == '__main__':
    main()

# Copy the Pre-Trained Model

Copy the pre-trained model from the train (results) folder to the desired folder to be used in the future.

In [ ]:
# OS (Operating system interfaces)
import os
# Shutil (High-level file operations)
import shutil

def main():
  # The specified path of the file.
  file_path_old = f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Result/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/train/weights/'
  file_path_new = f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Model/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/'
  
  # Name of the pre-trained model.
  model_name_old = 'best.pt'
  model_name_new = f'{CONST_YOLO_SIZE}_custom.pt'

  if os.path.isfile(file_path_new + model_name_old):
    print('[INFO] The file already exists..')
  else:
    # Copy the file.
    shutil.copy(file_path_old + model_name_old, file_path_new)
    print('[INFO] The file was successfully copied.')

    # Rename the file.
    os.rename(file_path_new + model_name_old, file_path_new + model_name_new)
    print('[INFO] The file was successfully renamed.')

if __name__ == '__main__':
    main()

# Convert Model to .onnx Format

Exporting the YOLOv8 model to a format that can be used for deployment. In this case, the model is converted to (*.onnx) format.

For more information see: https://docs.ultralytics.com/modes/export/

**Note:**

More information about the **onnx** format can be found at: https://onnx.ai

In [ ]:
# Ultralytics (Real-time object detection and image segmentation 
# model) [pip install ultralytics]
from ultralytics import YOLO

def main():
  # Load a pre-trained custom YOLO model.
  model = YOLO(f'{CONST_GOOGLE_DRIVE_ROOT}/YOLO/Model/Type_{CONST_DATASET_TYPE}_Obj_ID_{CONST_OBJECT_ID}/{CONST_YOLO_SIZE}_custom.pt')

  # Export the model to *.onnx format.
  model.export(format='onnx', opset=12)

if __name__ == '__main__':
    main()